## Deliverable 3. Create a Travel Itinerary Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import numpy as np

# Import API key
from config import google_api_key as g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Port Blair,IN,74.7,scattered clouds,11.6667,92.7500,Welcomhotel Bay Island Port Blair
1,2,Busselton,AU,68.0,broken clouds,-33.6500,115.3333,Observatory Guest House
2,6,Camacha,PT,60.8,few clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
3,7,Punta Arenas,CL,55.4,light rain,-53.1500,-70.9167,Hotel Dreams Del Estrecho
4,9,Bredasdorp,ZA,62.6,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor


In [5]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [6]:
# 4a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 4b. Display the figure
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [85]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City'] == "Sungaipenuh"]
vacation_end = vacation_df.loc[vacation_df['City'] == "Sungaipenuh"]
vacation_stop1 = vacation_df.loc[vacation_df['City'] == "Sibolga"]
vacation_stop2 = vacation_df.loc[vacation_df['City'] == "Meulaboh"] 
vacation_stop3 = vacation_df.loc[vacation_df['City'] == "Rantauprapat"] 

In [86]:
vacation_stop3

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
352,679,Rantauprapat,ID,74.7,broken clouds,2.1,99.8333,Permata Land Hotel & Resort


In [87]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start.to_numpy()
end = vacation_end.to_numpy()
stop1 = vacation_stop1.to_numpy()
stop2 = vacation_stop2.to_numpy()
stop3 = vacation_stop3.to_numpy()

In [88]:
start

array([[242, 'Sungaipenuh', 'ID', 64.99, 'overcast clouds', -2.0833,
        101.3833, 'Hotel Jaya Wisata 2']], dtype=object)

In [89]:
start[0][5]

-2.0833

In [90]:
lats = [start[0][5], stop1[0][5], stop2[0][5], stop3[0][5], end[0][5]]
lngs = [start[0][6], stop1[0][6], stop2[0][6], stop3[0][6], end[0][6]]
lats_lngs = zip(lats, lngs)

In [91]:
coords = list(lats_lngs)

In [92]:
len(coords)

5

In [93]:
coords

[(-2.0833, 101.3833),
 (1.7427, 98.7792),
 (4.1363, 96.1285),
 (2.1, 99.8333),
 (-2.0833, 101.3833)]

In [94]:
coordinates = []
for measure in coords:
    lat = measure[0]
    coordinates.append(lat)
    lng = measure[1]    
    coordinates.append(lng)
    
len(coordinates)
#why did I do this? the world may never know

10

In [95]:
coords[0]

(-2.0833, 101.3833)

In [96]:
coords[3]

(2.1, 99.8333)

In [99]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
vacay_roundtrip = gmaps.directions_layer(coords[0], coords[0],
                                        waypoints=[coords[1],coords[2],coords[3]],
                                         travel_mode="DRIVING"
                                        )
fig.add_layer(vacay_roundtrip)


fig


Figure(layout=FigureLayout(height='420px'))

In [100]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,242,Sungaipenuh,ID,64.99,overcast clouds,-2.0833,101.3833,Hotel Jaya Wisata 2
1,152,Sibolga,ID,77.22,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
2,325,Meulaboh,ID,74.77,scattered clouds,4.1363,96.1285,Kost Pria
3,679,Rantauprapat,ID,74.70,broken clouds,2.1000,99.8333,Permata Land Hotel & Resort


In [101]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [102]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))